In [3]:
# Imports
import numpy as np
import pandas as pd
import time
import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [4]:
# Defines
YEAR = 2017
THRESHOLD = .60
FILL_NA = 0
DATE_CONVERSION = 'timestamps'
PREPROCESSING = 'MinMax'
KFOLD_SPLITS = 10

In [5]:
# Functions
def check_na(train):
    # Finds the number of missing values in each column
    num_of_na = [train.loc[:,col].isnull().sum() for col in train]
    # Divide by rows for proportion 
    prop_na = [num / train.shape[0] for num in num_of_na]
    # Put the proporitons and column names into a df and sort
    na_df = pd.DataFrame({'prop_na' : prop_na, 'column' : train.columns}).sort_values('prop_na')
    return na_df

In [6]:
# Read csvs
if YEAR == 2016:
    properties = pd.read_csv('properties_2016.csv', low_memory = False)
    train = pd.read_csv('train_2016_v2.csv', low_memory = False)
elif YEAR == 2017:
    properties = pd.read_csv('properties_2017.csv', low_memory = False)
    train = pd.read_csv('train_2017.csv', low_memory = False)
# train has Y and properties has features
# Find row intersection of train and properties
train = train.merge(properties, on = 'parcelid', how = 'left')

In [7]:
print('BEFORE')
print(check_na(train))
# Remove all columns above the THRESHOLD
train = train.loc[:, (train.isnull().sum(axis=0) <= (train.shape[0]*THRESHOLD))]
print('AFTER')
print(check_na(train))

BEFORE
     prop_na                        column
0   0.000000                      parcelid
1   0.000000                      logerror
2   0.000000               transactiondate
27  0.000438                     longitude
34  0.000438     propertycountylandusecode
35  0.000438         propertylandusetypeid
37  0.000438        rawcensustractandblock
39  0.000438                regionidcounty
42  0.000438                       roomcnt
19  0.000438                          fips
7   0.000438                    bedroomcnt
6   0.000438                   bathroomcnt
54  0.000438                assessmentyear
26  0.000438                      latitude
53  0.000451             taxvaluedollarcnt
55  0.000464         landtaxvaluedollarcnt
56  0.000502                     taxamount
41  0.001082                   regionidzip
52  0.001920    structuretaxvaluedollarcnt
13  0.003028  calculatedfinishedsquarefeet
59  0.003621           censustractandblock
49  0.003917                     yearbuilt
21  

In [8]:

# Convert transactiondate strings into floats
date_strings = (train.values[:,2])
date_converted = []
if DATE_CONVERSION == 'timestamps':
    for string in date_strings:
        date_converted.append(time.mktime(datetime.datetime.strptime(string, "%Y-%m-%d").timetuple()))
train['transactiondate'] = np.asarray(date_converted)
# Drop the columns with string and int
train = train.drop(columns=['propertycountylandusecode', 'propertyzoningdesc'])

In [22]:
# Preprocessing

# scales a column x of a pandas dataframe
def minmaxscaler_dropna(x):
    # Formula from:
    # https://stackoverflow.com/questions/39758449/normalise-between-0-and-1-ignoring-nan
    return((x - x.min()) / (x.max() - x.min()))


if PREPROCESSING == 'MinMax':
#     scaler = MinMaxScaler()
#     scaler.fit(x)
#     x = scaler.transform(x)
    train2 = [minmaxscaler_dropna(train[col]) for col in train.columns]



In [26]:
type(train2)

list

In [29]:
train2 = pd.DataFrame(train2).transpose()
train2.head()

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,0.022842,0.471952,0.0,0.194444,0.2500,NaN,0.147059,0.083690,0.136324,0.297297,...,0.003280,0.000000,NaN,0.906250,0.042521,0.020837,NaN,0.010978,0.018741,5.196126e-04
1,0.040395,0.474979,0.0,0.055556,0.1250,NaN,0.000000,0.037649,0.061327,1.000000,...,0.003678,0.333333,NaN,0.744792,0.007701,0.009437,NaN,0.007678,0.009636,1.748451e-03
2,0.022133,0.469914,0.0,0.111111,0.1875,NaN,0.058824,0.031398,0.051144,0.297297,...,0.003609,0.400000,NaN,0.718750,0.007463,0.011492,NaN,0.009792,0.011027,5.186377e-04
3,0.009339,0.458945,0.0,0.166667,0.2500,0.636364,0.117647,0.063303,0.103115,0.000000,...,0.001146,0.000000,0.0,0.760417,0.009532,0.002938,NaN,0.000737,0.002996,4.708050e-06
4,0.001117,0.470071,0.0,0.166667,0.1875,0.636364,0.117647,0.033341,0.054309,0.000000,...,0.001544,0.000000,0.0,0.729167,0.006447,0.002414,NaN,0.000931,0.002581,5.321334e-07


In [30]:
y = train2.values[:,1]
y = y.reshape(y.shape[0],1)
x = train2.values[:,2:]

In [31]:
# x

In [32]:
# # scales a column x of a pandas dataframe
# def minmaxscaler_dropna(x):
#     # Formula from:
#     # https://stackoverflow.com/questions/39758449/normalise-between-0-and-1-ignoring-nan
#     (x - x.min()) / (x.max() - x.min())


# if PREPROCESSING == 'MinMax':
# #     scaler = MinMaxScaler()
# #     scaler.fit(x)
# #     x = scaler.transform(x)
#     x2 = [minmaxscaler_dropna(x.col) for col in x.columns]
# # x.col =     
# x2

In [33]:
# Replace all NAs with number defined in FILL_NA
train = train2.fillna(FILL_NA)

In [10]:
# KFolds
train_index_array = []
test_index_array = []
kf = KFold(n_splits = KFOLD_SPLITS, shuffle = True, random_state = 1)
for train_index, test_index in kf.split(x):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_index_array.append(train_index)
    test_index_array.append(test_index)
MY_INDEX = 6
x_train, x_test = x[train_index_array[MY_INDEX]], x[test_index_array[MY_INDEX]]
y_train, y_test = y[train_index_array[MY_INDEX]], y[test_index_array[MY_INDEX]]

TRAIN: [    0     2     3 ... 77609 77611 77612] TEST: [    1    28    30 ... 77602 77607 77610]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [   10    24    25 ... 77586 77593 77595]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    3    12    13 ... 77582 77603 77608]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    8    23    40 ... 77567 77597 77599]
TRAIN: [    0     1     3 ... 77610 77611 77612] TEST: [    2    11    22 ... 77569 77574 77605]
TRAIN: [    0     1     2 ... 77609 77610 77611] TEST: [   33    37    39 ... 77585 77601 77612]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [   26    35    36 ... 77587 77588 77591]
TRAIN: [    1     2     3 ... 77609 77610 77612] TEST: [    0     4     6 ... 77600 77606 77611]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    5    41    49 ... 77592 77604 77609]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    9    15    18 ... 77589 77594 77596]


In [11]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(9, input_dim = 27, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'linear')
    ])
sgd = tf.keras.optimizers.SGD(lr=0.1)
model.compile(loss = 'mse', optimizer = sgd)
model.fit(x_train, y_train, epochs = 500, batch_size = 32, verbose = 1)

Epoch 1/500
69852/69852 [==============================] - 3s 47us/step - loss: nan
Epoch 2/500
69852/69852 [==============================] - 2s 34us/step - loss: nan
Epoch 3/500
69852/69852 [==============================] - 3s 40us/step - loss: nan
Epoch 4/500
69852/69852 [==============================] - 3s 39us/step - loss: nan
Epoch 5/500
69852/69852 [==============================] - 3s 37us/step - loss: nan
Epoch 6/500
69852/69852 [==============================] - 3s 41us/step - loss: nan
Epoch 7/500
69852/69852 [==============================] - 3s 38us/step - loss: nan
Epoch 8/500
69852/69852 [==============================] - 3s 40us/step - loss: nan
Epoch 9/500
69852/69852 [==============================] - 2s 32us/step - loss: nan
Epoch 10/500
69852/69852 [==============================] - 2s 29us/step - loss: nan
Epoch 11/500
69852/69852 [==============================] - 2s 32us/step - loss: nan
Epoch 12/500
69852/69852 [==============================] - 2s 29us/step -

69852/69852 [==============================] - 2s 26us/step - loss: nan
Epoch 98/500
69852/69852 [==============================] - 2s 26us/step - loss: nan
Epoch 99/500
69852/69852 [==============================] - 2s 30us/step - loss: nan
Epoch 100/500
69852/69852 [==============================] - 2s 29us/step - loss: nan
Epoch 101/500
69852/69852 [==============================] - 2s 35us/step - loss: nan
Epoch 102/500
69852/69852 [==============================] - 3s 38us/step - loss: nan
Epoch 103/500
69852/69852 [==============================] - 2s 31us/step - loss: nan
Epoch 104/500
69852/69852 [==============================] - 2s 24us/step - loss: nan
Epoch 105/500
69852/69852 [==============================] - 2s 24us/step - loss: nan
Epoch 106/500
69852/69852 [==============================] - 2s 28us/step - loss: nan
Epoch 107/500
69852/69852 [==============================] - 2s 28us/step - loss: nan
Epoch 108/500
69852/69852 [==============================] - 2s 24us/s

69852/69852 [==============================] - 3s 43us/step - loss: nan
Epoch 193/500
69852/69852 [==============================] - 3s 50us/step - loss: nan
Epoch 194/500
69852/69852 [==============================] - 2s 33us/step - loss: nan
Epoch 195/500
69852/69852 [==============================] - 2s 30us/step - loss: nan
Epoch 196/500
69852/69852 [==============================] - 4s 54us/step - loss: nan
Epoch 197/500
69852/69852 [==============================] - 4s 56us/step - loss: nan
Epoch 198/500
69852/69852 [==============================] - 3s 39us/step - loss: nan
Epoch 199/500
69852/69852 [==============================] - 4s 56us/step - loss: nan
Epoch 200/500
69852/69852 [==============================] - 4s 62us/step - loss: nan
Epoch 201/500
69852/69852 [==============================] - 3s 49us/step - loss: nan
Epoch 202/500
69852/69852 [==============================] - 3s 46us/step - loss: nan
Epoch 203/500
69852/69852 [==============================] - 3s 42us

69852/69852 [==============================] - 4s 61us/step - loss: nan
Epoch 288/500
69852/69852 [==============================] - 4s 54us/step - loss: nan
Epoch 289/500
69852/69852 [==============================] - 3s 37us/step - loss: nan 
Epoch 290/500
69852/69852 [==============================] - 3s 45us/step - loss: nan
Epoch 291/500
69852/69852 [==============================] - 3s 50us/step - loss: nan
Epoch 292/500
69852/69852 [==============================] - 3s 45us/step - loss: nan
Epoch 293/500
69852/69852 [==============================] - 4s 52us/step - loss: nan
Epoch 294/500
69852/69852 [==============================] - 4s 62us/step - loss: nan
Epoch 295/500
69852/69852 [==============================] - 4s 51us/step - loss: nan
Epoch 296/500
69852/69852 [==============================] - 3s 43us/step - loss: nan
Epoch 297/500
69852/69852 [==============================] - 3s 36us/step - loss: nan
Epoch 298/500
69852/69852 [==============================] - 3s 42u

69852/69852 [==============================] - 2s 32us/step - loss: nan
Epoch 383/500
69852/69852 [==============================] - 2s 28us/step - loss: nan
Epoch 384/500
69852/69852 [==============================] - 2s 27us/step - loss: nan
Epoch 385/500
69852/69852 [==============================] - 2s 29us/step - loss: nan
Epoch 386/500
69852/69852 [==============================] - 3s 46us/step - loss: nan
Epoch 387/500
69852/69852 [==============================] - 2s 30us/step - loss: nan
Epoch 388/500
69852/69852 [==============================] - 2s 26us/step - loss: nan
Epoch 389/500
69852/69852 [==============================] - 2s 29us/step - loss: nan
Epoch 390/500
69852/69852 [==============================] - 2s 31us/step - loss: nan
Epoch 391/500
69852/69852 [==============================] - 2s 27us/step - loss: nan
Epoch 392/500
69852/69852 [==============================] - 2s 25us/step - loss: nan
Epoch 393/500
69852/69852 [==============================] - 2s 26us

69852/69852 [==============================] - 3s 41us/step - loss: nan
Epoch 478/500
69852/69852 [==============================] - 2s 33us/step - loss: nan
Epoch 479/500
69852/69852 [==============================] - 2s 28us/step - loss: nan
Epoch 480/500
69852/69852 [==============================] - 2s 26us/step - loss: nan
Epoch 481/500
69852/69852 [==============================] - 2s 26us/step - loss: nan
Epoch 482/500
69852/69852 [==============================] - 2s 25us/step - loss: nan
Epoch 483/500
69852/69852 [==============================] - 2s 27us/step - loss: nan
Epoch 484/500
69852/69852 [==============================] - 2s 26us/step - loss: nan
Epoch 485/500
69852/69852 [==============================] - 2s 26us/step - loss: nan
Epoch 486/500
69852/69852 [==============================] - 2s 26us/step - loss: nan
Epoch 487/500
69852/69852 [==============================] - 2s 29us/step - loss: nan
Epoch 488/500
69852/69852 [==============================] - 2s 27us

In [ ]:
model.evaluate(x_test, y_test, verbose = 1)